In [22]:
import numpy as np

def generate_random_problem(m, n, rho):
    # Generate random matrix B
    B = np.random.uniform(-1, 1, (m, n))
    
    # Generate random vector v*
    v_star = np.random.uniform(0, 1, m)
    y_star = v_star / np.linalg.norm(v_star)
    
    # Calculate ai coefficients
    B_transpose_y_star = np.abs(np.dot(B.T, y_star))
    sorted_indices = np.argsort(-B_transpose_y_star)  # Sort descending by absolute value
    alpha = np.zeros(n)
    m_star = int(m * 0.8)  # Assume m_star as 80% of m

    for i in range(n):
        index = sorted_indices[i]
        b_i_y_star = B_transpose_y_star[index]
        if i < m_star:
            alpha[index] = 1 / b_i_y_star if b_i_y_star != 0 else 0
        elif b_i_y_star <= 0.1:
            alpha[index] = 1
        else:
            xi = np.random.uniform(0, 1)
            alpha[index] = xi / b_i_y_star

    # Create matrix A from B and alpha
    A = B * alpha  # element-wise multiplication for each column

    # Generate the components of the primal solution x*
    x_star = np.zeros(n)
    xi_values = np.random.uniform(0, rho / np.sqrt(m_star), m_star)
    for i in range(m_star):
        index = sorted_indices[i]
        x_star[index] = xi_values[i] * np.sign(np.dot(A[:, index], y_star))

    # Define vector b
    b = y_star + np.dot(A, x_star)

    return A, b, x_star

def main():
    m = 100
    n = 150
    rho = 0.5

    A, b, x_star = generate_random_problem(m, n, rho)

    print("Matrix A:", A)
    print("Vector b:", b)
    print("Optimal solution x*:", x_star)

    # Placeholder for optimization methods
    # You can implement specific optimization methods here

if __name__ == "__main__":
    main()


Matrix A: [[-0.74225241 -1.4221677   0.81161916 ...  0.9278028   1.28919541
  -0.86090645]
 [ 2.43743206  1.7875366  -0.22903614 ...  0.55143069  1.2985
   1.04128406]
 [ 1.29054691  1.18093008  0.0648249  ...  0.62029703 -0.27334158
  -0.56098982]
 ...
 [-0.84777813 -1.77531103  0.38568661 ... -0.57551962  0.2164996
   0.20797018]
 [ 2.05773278  1.09012076 -0.01015446 ...  0.67977832 -1.36319428
  -1.44487161]
 [ 0.0120635   2.0104514   0.14833699 ... -0.74912237 -0.66107112
  -0.46020108]]
Vector b: [ 0.25516312  0.68845172  0.32270935  0.61596866  0.32955678 -0.02264979
  0.03802707  0.00937889  0.31621405  0.60419264  0.53395419  0.25749019
  0.10523586  0.05324423  0.36847392  0.1165483   0.15283683  0.25167111
  0.22702027 -0.01716472  0.63143738  0.10650085  0.55841735  0.3542849
  0.84702772  0.25615724  0.7599067   0.61697083 -0.43662154 -0.01808373
  0.59862704  0.7626474  -0.04453468  0.57547957  0.09104971 -0.02908373
 -0.14006373  0.07391006  0.31622348  0.8966511   0.0047

In [23]:
x_star

array([ 0.        ,  0.        ,  0.25366027,  0.        , -0.09028216,
        0.09119534, -0.20380756,  0.40777997, -0.01892502,  0.97640837,
       -0.90103861,  0.        , -0.27547684,  0.27880742,  0.        ])

In [20]:
from sklearn.model_selection import train_test_split
import numpy as np

def generate_random_problem(m, n, rho):
    B = np.random.uniform(-1, 1, (m, n))
    v_star = np.random.uniform(0, 1, m)
    y_star = v_star / np.linalg.norm(v_star)
    B_transpose_y_star = np.abs(np.dot(B.T, y_star))
    sorted_indices = np.argsort(-B_transpose_y_star)
    alpha = np.zeros(n)
    m_star = int(m * 0.8)

    for i in range(n):
        index = sorted_indices[i]
        b_i_y_star = B_transpose_y_star[index]
        if i < m_star:
            alpha[index] = 1 / b_i_y_star if b_i_y_star != 0 else 0
        elif b_i_y_star <= 0.1:
            alpha[index] = 1
        else:
            xi = np.random.uniform(0, 1)
            alpha[index] = xi / b_i_y_star

    A = B * alpha
    x_star = np.zeros(n)
    xi_values = np.random.uniform(0, rho / np.sqrt(m_star), m_star)
    for i in range(m_star):
        index = sorted_indices[i]
        x_star[index] = xi_values[i] * np.sign(np.dot(A[:, index], y_star))

    b = y_star + np.dot(A, x_star)
    return A, b, x_star

def main():
    m = 100  # Number of equations (rows in A)
    n = 150  # Number of variables (columns in A)
    rho = 0.5

    A, b, x_star = generate_random_problem(m, n, rho)

    # Split data into training and testing sets, each column of A matches one element of b
    A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.2, random_state=42)

    print("Training set A shape:", A_train.shape)
    print("Testing set A shape:", A_test.shape)
    print("Training target b shape:", b_train.shape)
    print("Testing target b shape:", b_test.shape)

if __name__ == "__main__":
    main()


Training set A shape: (80, 150)
Testing set A shape: (20, 150)
Training target b shape: (80,)
Testing target b shape: (20,)


In [21]:
import numpy as np

def generate_sparse_least_squares(m, n, rho):
    """
    Generates a Sparse Least Squares problem.
    
    Parameters:
    m (int): Number of rows in matrix A.
    n (int): Number of columns in matrix A, n > m.
    rho (float): Parameter determining the sparsity and size of the components of the optimal solution x*.
    
    Returns:
    A (numpy.ndarray): Generated dense matrix A of shape (m, n).
    b (numpy.ndarray): Generated vector b of shape (m,).
    x_star (numpy.ndarray): Sparse optimal solution x* of shape (n,).
    """
    # Generate a dense matrix A with random values
    A = np.random.uniform(-1, 1, (m, n))
    
    # Generate a random sparse vector x_star with exactly m non-zero entries
    x_star = np.zeros(n)
    non_zero_indices = np.random.choice(n, m, replace=False)
    x_star[non_zero_indices] = np.random.uniform(-rho, rho, m)
    
    # Compute the vector b
    b = A @ x_star + np.random.normal(0, 0.1, m)  # Adding some noise to b

    return A, b, x_star

# Example usage
m, n, rho = 10, 15, 1.0  # m < n for underdetermined system
A, b, x_star = generate_sparse_least_squares(m, n, rho)
print("Matrix A:\n", A)
print("Vector b:\n", b)
print("Sparse solution x*:\n", x_star)


Matrix A:
 [[ 0.5481599  -0.27874607  0.42141924  0.0359002   0.71130433 -0.68969982
   0.41134356 -0.37394426  0.40217497 -0.9510156  -0.75838798 -0.80535468
  -0.97083868  0.95218043 -0.30729731]
 [-0.71626615 -0.11785537 -0.76079833 -0.60720566 -0.63082371 -0.77005534
   0.41524066  0.55626184 -0.38520624 -0.12633922  0.33620783 -0.45422986
   0.5804055   0.47527683 -0.9817371 ]
 [ 0.68880929 -0.31541617 -0.72124398  0.93719637 -0.0989078  -0.11906775
  -0.59893527 -0.72076539  0.57862614 -0.59022755  0.03507602  0.21354169
   0.43412918  0.3468911   0.76217326]
 [ 0.67222488  0.41759852 -0.01163454 -0.58873854 -0.16949829  0.44534996
  -0.02373823 -0.13653765 -0.31227751 -0.77417156  0.244015   -0.32968259
  -0.14797736 -0.63027559  0.56423636]
 [ 0.17373103 -0.8714035   0.04087582 -0.76393758  0.14715549  0.51129093
  -0.28376791  0.72282905 -0.09513624 -0.26055432  0.36538397  0.00943211
   0.51903013  0.73015935  0.13738513]
 [-0.83575139  0.4675956  -0.06049348  0.11135851 -0.8